## Movie Search

In [1]:
import zipfile
import json
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import helpers

import boto3
from elasticsearch import RequestsHttpConnection
from requests_aws4auth import AWS4Auth

### AWS elasticsearch

In [62]:
# Local Elasticsearch
# Use this or else AWS blocks below
# connects to port 9300 by default
es = Elasticsearch()

In [8]:
# AWS Elasticsearch
# Don't run this if local version is being used
# Authorization
# If we have an open policy on AWS ES this block is not needed and http_auth must be commented out below
region = 'us-west-2' 
service = 'es'
#Credentials calls the user keys identified from aws-cli command aws configure
credentials = boto3.Session().get_credentials()
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, service, session_token=credentials.token)

#Create ES Connection with AWS
#AWS ES endpoint link
host = 'put your aws elasticsearch endpoint here'

es = Elasticsearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)
print(es.info())

{'name': 'MJtp04b', 'cluster_name': '634195893235:movies', 'cluster_uuid': 'ymqDfLcWQkGDXG-otApe-A', 'version': {'number': '6.3.1', 'build_flavor': 'oss', 'build_type': 'zip', 'build_hash': 'eb782d0', 'build_date': '2018-09-11T14:05:25.216906Z', 'build_snapshot': False, 'lucene_version': '7.3.1', 'minimum_wire_compatibility_version': '5.6.0', 'minimum_index_compatibility_version': '5.0.0'}, 'tagline': 'You Know, for Search'}


In [63]:
#Check Cluster Health
es.cluster.health()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 26,
 'active_shards': 26,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 25,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 50.98039215686274}

In [10]:
# Index Info
es.cat.indices()

'yellow open df1     f9yaHDIhSpqrK1a_GGX6pQ 5 1 1000 0  1.2mb  1.2mb\nyellow open df4     jnSJXJxJQKivv9eiivS9Tg 5 1 1000 0  1.3mb  1.3mb\nyellow open df3     hMg5Tg_5T8mTEL0nhq7DEg 5 1 1000 0  1.2mb  1.2mb\nyellow open df2     s29bB2JnQOi37AdW5HDbmg 5 1 1000 0  1.2mb  1.2mb\nyellow open df5     FEr4PYfsR1G8lUYxGmwU9A 5 1 1000 0  1.4mb  1.4mb\ngreen  open .kibana WbySMBDlS5qZ8rzIJPm96A 1 0    6 0 33.2kb 33.2kb\nyellow open movies  LDYQOMNNQhOgpiJ2hsN6NQ 5 1 5000 0  5.4mb  5.4mb\n'

### Search Elasticsearch Cluster

In [51]:
# Search Result Dictionary
search_dict = es.search(index = ['df1','df2','df3','df3', 'df5'], q='thor')

In [52]:
len(search_dict['hits']['hits'])

7

In [55]:
search_list = [k['_source'] for k in search_dict['hits']['hits']]

In [58]:
df_search = pd.DataFrame(search_list, columns=search_list[0].keys())
df_search

,title,year,rating,rank,genres,plot,directors,actors,release_date,running_time_secs,image_url
0,Thor,2011,7.0,135,"[Action, Adventure, Fantasy]",The powerful but arrogant god Thor is cast out...,"[Kenneth Branagh, Joss Whedon]","[Chris Hemsworth, Anthony Hopkins, Natalie Por...",2011-04-21,6900,http://ia.media-imdb.com/images/M/MV5BMTYxMjA5...
1,Vikingdom,2013,5.0,602,[Adventure],"A forgotten king, Eirick, is tasked with the i...",[Yusry Abd Halim],"[Dominic Purcell, Natassia Malthe, Conan Stevens]",2013-09-12,0,http://ia.media-imdb.com/images/M/MV5BMTYxODY5...
2,Percy Jackson: Sea of Monsters,2013,6.5,85,"[Adventure, Family, Fantasy]","In order to restore their dying safe haven, th...",[Thor Freudenthal],"[Logan Lerman, Alexandra Daddario, Brandon T. ...",2013-08-07,6360,http://ia.media-imdb.com/images/M/MV5BMTUwODUy...
3,Diary of a Wimpy Kid,2010,6.1,1665,"[Comedy, Family]",Live-action adaptation of Jeff Kinney's illust...,[Thor Freudenthal],"[Zachary Gordon, Robert Capron, Rachael Harris]",2010-03-18,5640,http://ia.media-imdb.com/images/M/MV5BMTg3NzQ2...
4,Hotel for Dogs,2009,5.1,4536,"[Comedy, Family]",Two kids secretly take in stray dogs at a vaca...,[Thor Freudenthal],"[Emma Roberts, Jake T. Austin, Lisa Kudrow]",2009-01-15,6000,http://ia.media-imdb.com/images/M/MV5BMTI5Njkx...
5,Kon-Tiki,2012,7.2,1026,"[Adventure, History]","Legendary explorer Thor Heyerdal's epic 4,300-...","[Joachim Rønning, Espen Sandberg]","[Pål Sverre Hagen, Anders Baasmo Christiansen,...",2012-08-18,7080,http://ia.media-imdb.com/images/M/MV5BMjMxOTU2...
6,Thor: The Dark World,2013,0.0,5,"[Action, Adventure, Fantasy]",Faced with an enemy that even Odin and Asgard ...,[Alan Taylor],"[Chris Hemsworth, Natalie Portman, Tom Hiddles...",2013-10-30,0,http://ia.media-imdb.com/images/M/MV5BMTQyNzAw...
